# D16 Ví dụ: Data dictionary

## Mục đích

Áp dụng tất cả những kiến thức đã học về Pandas để xây dựng một công cụ rất quan trọng trong quản lí số liệu: từ điển số liệu (data dictionary, codebook).


## Bài toán

Thông thường khi nhập liệu và xuất số liệu để phân tích, số liệu của bạn sẽ nằm ở một trong hai định dạng: mã hóa (code) hoặc dán nhãn (label). Những người làm phân tích số liệu chuyên nghiệp thường không thích số liệu dán nhãn vì có thể họ sẽ muốn điều chỉnh số liệu trước khi phân tích (chẳng hạn, nhóm trình độ học vấn "Cấp 1", "Cấp 2" và "Cấp 3" thành "Từ cấp 3 trở xuống"). Ngoài ra, tên biến cũng là một vấn đề: tên biến code là "hads_02" nhưng label sẽ là "2. Tôi vẫn có hứng thú với những thứ từng có hứng thú"; nếu chúng ta để tên biến dưới dạng label thì nó sẽ rất phiền khi phân tích số liệu. Tóm lại, khi xuất số liệu ra chúng ta nên để dưới dạng code.

Điều này dẫn đến việc bạn sẽ phải label lại các biến khi bạn phân tích vì kết quả phân tích không thể để dưới dạng code và gửi đi cho nhóm nghiên cứu hoặc nhà tài trợ. Tuy nhiên, nếu label biến bằng tay, bạn sẽ mất rất nhiều thời gian và khi label thay đổi, bạn sẽ phải lần mò trong các đoạn code để sửa lại. Do vậy, chúng ta cần tìm cách tự động hóa công việc này. Cách đơn giản nhất là sử dụng data dictionary.

Quan sát bộ số liệu sau.

In [1]:
import pandas as pd
import numpy as np

np.random.seed(0)
n = 500

d = pd.DataFrame({
    "id": range(1, n + 1),
    "sex": np.random.choice([1, 2], n, replace=True),
    "age": np.random.randint(20, 60, n),
    "edu": np.random.choice([1, 2, 3], n, replace=True, p=[0.2, 0.5, 0.3]),
    "bmi": np.random.normal(21.5, 1.5, n),
    "tc_1_tngc": np.random.choice([0, 1], n, replace=True, p=[0.35, 0.65]),
    "tc_2_nrat": np.random.choice([0, 1], n, replace=True, p=[0.25, 0.75]),
    "tc_3_dnguc": np.random.choice([0, 1], n, replace=True, p=[0.05, 0.95]),
})

d.head()

,id,sex,age,edu,bmi,tc_1_tngc,tc_2_nrat,tc_3_dnguc
0,1,1,38,3,18.411116,1,1,1
1,2,2,43,3,21.438402,1,1,0
2,3,2,21,2,20.884253,1,1,1
3,4,1,26,3,20.912275,1,1,1
4,5,2,50,1,20.355965,1,1,1


Bạn có thể thấy rằng biến `"sex"` và `"edu"` là biến danh mục (categorical) và đang để ở dạng code. Chúng ta sẽ cần dán nhãn cho nó khi phân tích số liệu. Với những kiến thức đã học, bạn biết rằng chúng ta có thể dùng hàm `replace()` để làm việc này; ví dụ, chúng ta có thể xây dựng một từ điển như sau.

In [2]:
data_dict = {
    "sex": {1: "Nam", 2: "Nữ"},
    "edu": {1: "Cấp 2 trở xuống", 2: "Cấp 3", 3: "Đại học trở lên"}
}

d.replace(data_dict).head()

,id,sex,age,edu,bmi,tc_1_tngc,tc_2_nrat,tc_3_dnguc
0,1,Nam,38,Đại học trở lên,18.411116,1,1,1
1,2,Nữ,43,Đại học trở lên,21.438402,1,1,0
2,3,Nữ,21,Cấp 3,20.884253,1,1,1
3,4,Nam,26,Đại học trở lên,20.912275,1,1,1
4,5,Nữ,50,Cấp 2 trở xuống,20.355965,1,1,1


Hoặc trong một số trường hợp bạn muốn làm như vậy khi phân tích số liệu.

In [3]:
d.set_index("id").groupby(["sex", "edu"])[["age", "bmi"]] \
    .agg(["mean", "std"]) \
    .stack(0).apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1) \
    .unstack(-1).reset_index() \
    .replace(data_dict)

,sex,edu,age,bmi
0,Nam,Cấp 2 trở xuống,37.36 (11.70),21.73 (1.43)
1,Nam,Cấp 3,41.30 (11.84),21.63 (1.60)
2,Nam,Đại học trở lên,39.23 (10.98),21.34 (1.81)
3,Nữ,Cấp 2 trở xuống,37.62 (11.47),21.38 (1.37)
4,Nữ,Cấp 3,39.47 (11.82),21.56 (1.46)
5,Nữ,Đại học trở lên,39.49 (11.88),21.52 (1.59)


Nhưng câu hỏi đặt ra là chúng ta sẽ xây dựng từ điển này như thế nào? Thông thường, chúng ta sẽ lưu nó vào trong một sheet trong Excel hoặc các công cụ bảng tính khác như thế này.

In [4]:
d_codes = pd.concat(list(map(
    lambda x: pd.DataFrame([{"list_name": x, "code": code, "label": value}
        for code, value in data_dict[x].items()]),
    data_dict.keys()
)))

d_codes

,list_name,code,label
0,sex,1,Nam
1,sex,2,Nữ
0,edu,1,Cấp 2 trở xuống
1,edu,2,Cấp 3
2,edu,3,Đại học trở lên


Bạn có thể thấy trong bộ số liệu ở trên của chúng ta, có 3 trường triệu chứng là biến nhị phân (binary). Thông thường chúng ta sẽ mã hóa chung các biến nhị phân giống nhau (ví dụ, 1=Có, 0=Không). Do đó, tất cả các trường này đều chỉ cần một danh sách code.

In [5]:
d_codes = pd.concat([d_codes,
    pd.DataFrame({
        "list_name": "yesno",
        "code": [0, 1],
        "label": ["Không", "Có"],
    })
])

d_codes

,list_name,code,label
0,sex,1,Nam
1,sex,2,Nữ
0,edu,1,Cấp 2 trở xuống
1,edu,2,Cấp 3
2,edu,3,Đại học trở lên
0,yesno,0,Không
1,yesno,1,Có


Trong trường hợp này, bạn sẽ cần thêm một "từ điển" nữa, ghi danh sách các biến và tên "code list" tương ứng của chúng.

In [6]:
d_vars = pd.DataFrame({
    "var": d.columns,
    "label": ["STT", "Giới", "Tuổi", "Trình độ học vấn",
        "BMI", "Trào ngược", "Nóng rát", "Đau ngực"],
    "list_name": [np.nan, "sex", np.nan, "edu",
        np.nan, "yesno", "yesno", "yesno"],
})

d_vars

,var,label,list_name
0,id,STT,NaN
1,sex,Giới,sex
2,age,Tuổi,NaN
3,edu,Trình độ học vấn,edu
4,bmi,BMI,NaN
5,tc_1_tngc,Trào ngược,yesno
6,tc_2_nrat,Nóng rát,yesno
7,tc_3_dnguc,Đau ngực,yesno


Với ba bảng `d_vars`, `d_codes`, và `d`, chúng ta đã sẵn sàng cho giải pháp về data dictionary.

## Giải pháp

### Xây dựng từ điển từ `d_codes`

Việc đầu tiên, chúng ta sẽ xây dựng từ điển từ các danh sách mã trong `d_codes`. Từ điển này chưa thể dùng cho hàm `replace()` được ngay vì nó chứa danh sách code và label theo `"list_name"`, không phải theo tên biến.

Chúng ta sẽ xây dựng một hàm tên là `make_dict()` để tạo từ điển từ các cột trong data frame. Chúng ta sẽ dùng nó nhiều lần.

In [7]:
def make_dict(d: pd.DataFrame, key_col: str, value_col: str) -> dict:
    return dict(zip(d[key_col], d[value_col]))

make_dict(d_codes.query("list_name == 'yesno'"), "code", "label")

{0: 'Không', 1: 'Có'}

Như bạn thấy, chúng ta có thể dễ dàng tạo ra một từ điển cho từng `"list_name"`. Bây giờ làm thế nào để tạo ra từ điển cho tất cả các danh sách này? Nếu bạn còn nhớ hàm `groupby()` của Pandas, đây sẽ là công cụ hữu dụng.

In [8]:
def make_codes_dict(d: pd.DataFrame, listname_col: str, key_col: str, value_col: str) -> dict[str, dict]:
    d_grpby = d.groupby(listname_col)
    return {
        list_name: make_dict(d_grpby.get_group(list_name), key_col, value_col)
        for list_name in d_grpby.groups
    }

codes_dict = make_codes_dict(d_codes, "list_name", "code", "label")
codes_dict

{'edu': {1: 'Cấp 2 trở xuống', 2: 'Cấp 3', 3: 'Đại học trở lên'},
 'sex': {1: 'Nam', 2: 'Nữ'},
 'yesno': {0: 'Không', 1: 'Có'}}

### Xây dựng từ điển cho hàm `replace()`

Công việc này sẽ gồm hai bước. Bước đầu tiên, chúng ta sẽ lọc ra các biến nào là biến danh mục (có trường `"list_name"` không phải NA) để xây dựng một từ điển từ `"var"` sang `"list_name"`, sau đó bước hai là dùng từ điển này để xây dựng từ điển cho hàm `replace()`.

In [9]:
vars_dict = make_dict(d_vars.query("list_name.notna()"), "var", "list_name")
vars_dict

{'sex': 'sex',
 'edu': 'edu',
 'tc_1_tngc': 'yesno',
 'tc_2_nrat': 'yesno',
 'tc_3_dnguc': 'yesno'}

In [10]:
replace_dict = {v: codes_dict[list_name] for v, list_name in vars_dict.items()}
replace_dict

{'sex': {1: 'Nam', 2: 'Nữ'},
 'edu': {1: 'Cấp 2 trở xuống', 2: 'Cấp 3', 3: 'Đại học trở lên'},
 'tc_1_tngc': {0: 'Không', 1: 'Có'},
 'tc_2_nrat': {0: 'Không', 1: 'Có'},
 'tc_3_dnguc': {0: 'Không', 1: 'Có'}}

Hãy cùng xem kết quả.

In [11]:
d_labeled = d.replace(replace_dict)
d_labeled.head()

,id,sex,age,edu,bmi,tc_1_tngc,tc_2_nrat,tc_3_dnguc
0,1,Nam,38,Đại học trở lên,18.411116,Có,Có,Có
1,2,Nữ,43,Đại học trở lên,21.438402,Có,Có,Không
2,3,Nữ,21,Cấp 3,20.884253,Có,Có,Có
3,4,Nam,26,Đại học trở lên,20.912275,Có,Có,Có
4,5,Nữ,50,Cấp 2 trở xuống,20.355965,Có,Có,Có


In [12]:
var_label_dict = make_dict(d_vars, "var", "label")
var_label_dict

{'id': 'STT',
 'sex': 'Giới',
 'age': 'Tuổi',
 'edu': 'Trình độ học vấn',
 'bmi': 'BMI',
 'tc_1_tngc': 'Trào ngược',
 'tc_2_nrat': 'Nóng rát',
 'tc_3_dnguc': 'Đau ngực'}

In [13]:
d.set_index("id").groupby(["sex", "edu"])[["age", "bmi"]] \
    .agg(["mean", "std"]) \
    .stack(0).apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1) \
    .unstack(-1).reset_index() \
    .replace(data_dict).rename(columns=var_label_dict)

,Giới,Trình độ học vấn,Tuổi,BMI
0,Nam,Cấp 2 trở xuống,37.36 (11.70),21.73 (1.43)
1,Nam,Cấp 3,41.30 (11.84),21.63 (1.60)
2,Nam,Đại học trở lên,39.23 (10.98),21.34 (1.81)
3,Nữ,Cấp 2 trở xuống,37.62 (11.47),21.38 (1.37)
4,Nữ,Cấp 3,39.47 (11.82),21.56 (1.46)
5,Nữ,Đại học trở lên,39.49 (11.88),21.52 (1.59)


## Tạo biến categorical

Sau khi `replace()`, bạn có thể sẽ muốn chuyển các biến này thành dạng categorical để tiết kiệm không gian bộ nhớ máy tính và tối ưu hóa các tính toán. Bạn có thể làm như sau.

In [14]:
d_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          500 non-null    int64  
 1   sex         500 non-null    object 
 2   age         500 non-null    int32  
 3   edu         500 non-null    object 
 4   bmi         500 non-null    float64
 5   tc_1_tngc   500 non-null    object 
 6   tc_2_nrat   500 non-null    object 
 7   tc_3_dnguc  500 non-null    object 
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 29.4+ KB


In [15]:
vars_cat = list(replace_dict.keys())
d_labeled[vars_cat] = d_labeled[vars_cat].astype("category")

d_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          500 non-null    int64   
 1   sex         500 non-null    category
 2   age         500 non-null    int32   
 3   edu         500 non-null    category
 4   bmi         500 non-null    float64 
 5   tc_1_tngc   500 non-null    category
 6   tc_2_nrat   500 non-null    category
 7   tc_3_dnguc  500 non-null    category
dtypes: category(5), float64(1), int32(1), int64(1)
memory usage: 12.9 KB


Đối với biến `"edu"`, bạn có thể muốn thiết lập dưới dạng biến thứ hạng (ordinal) thay vì định danh (nominal). Tương tự, đối với các biến nhị phân, bạn muốn hiển thị kết quả "Có" trước "Không". Trong trường hợp này, chúng ta cần thiết lập thêm một thông số cho từ điển `d_vars` để tạo ra các trường categorical có thứ tự (ordered).

In [16]:
d_vars["order"] = [np.nan, np.nan, np.nan, "1 2 3", np.nan, "1 0", "1 0", "1 0"]
d_vars

,var,label,list_name,order
0,id,STT,NaN,NaN
1,sex,Giới,sex,NaN
2,age,Tuổi,NaN,NaN
3,edu,Trình độ học vấn,edu,1 2 3
4,bmi,BMI,NaN,NaN
5,tc_1_tngc,Trào ngược,yesno,1 0
6,tc_2_nrat,Nóng rát,yesno,1 0
7,tc_3_dnguc,Đau ngực,yesno,1 0


Việc đầu tiên là cần tạo ra danh sách các code dưới dạng số từ các chuỗi kí tự trong trường `"order"`.

In [17]:
d_hasorder = d_vars.query("order.notna()")
ordered_dict = {
    v: [int(s) for s in order.split()]
    for v, order in zip(d_hasorder["var"], d_hasorder["order"])
}
ordered_dict

{'edu': [1, 2, 3],
 'tc_1_tngc': [1, 0],
 'tc_2_nrat': [1, 0],
 'tc_3_dnguc': [1, 0]}

Sau đó, chúng ta có thể tra cứu label tương ứng của các code này.

In [18]:
{
    v: [replace_dict[v][code] for code in codes_order]
    for v, codes_order in ordered_dict.items()
}

{'edu': ['Cấp 2 trở xuống', 'Cấp 3', 'Đại học trở lên'],
 'tc_1_tngc': ['Có', 'Không'],
 'tc_2_nrat': ['Có', 'Không'],
 'tc_3_dnguc': ['Có', 'Không']}

Vậy chúng ta có thể kết hợp hai đoạn lệnh trên vào một hàm.

In [19]:
def make_order_dict(d: pd.DataFrame, var_col: str, order_col: str, replace_dict: dict) -> dict[str, list]:
    return {
        v: [replace_dict[v][int(s)] for s in order.split()]
        for v, order in zip(d[var_col], d_hasorder[order_col])
    }

order_dict = make_order_dict(d_vars.query("order.notna()"), "var", "order", replace_dict)
order_dict

{'edu': ['Cấp 2 trở xuống', 'Cấp 3', 'Đại học trở lên'],
 'tc_1_tngc': ['Có', 'Không'],
 'tc_2_nrat': ['Có', 'Không'],
 'tc_3_dnguc': ['Có', 'Không']}

In [20]:
for v, order in order_dict.items():
    d_labeled[v] = d_labeled[v].cat.set_categories(order, ordered=True)

d_labeled["edu"].head()

0    Đại học trở lên
1    Đại học trở lên
2              Cấp 3
3    Đại học trở lên
4    Cấp 2 trở xuống
Name: edu, dtype: category
Categories (3, object): ['Cấp 2 trở xuống' < 'Cấp 3' < 'Đại học trở lên']

In [21]:
from tableone import TableOne

TableOne(d_labeled.set_index("id"), missing=False, rename=var_label_dict)

Overall
n                                                500
Giới, n (%)             Nam               238 (47.6)
                        Nữ                262 (52.4)
Tuổi, mean (SD)                          39.5 (11.7)
Trình độ học vấn, n (%) Cấp 2 trở xuống    97 (19.4)
                        Cấp 3             253 (50.6)
                        Đại học trở lên   150 (30.0)
BMI, mean (SD)                            21.5 (1.6)
Trào ngược, n (%)       Có                342 (68.4)
                        Không             158 (31.6)
Nóng rát, n (%)         Có                380 (76.0)
                        Không             120 (24.0)
Đau ngực, n (%)         Có                472 (94.4)
                        Không               28 (5.6)

## Tổng kết toàn bộ quá trình

Nếu bạn xác định sẽ làm tất cả những thao tác trên, chúng ta có thể tổng hợp lại như sau.

In [22]:
def make_dict(d: pd.DataFrame, key_col: str, value_col: str) -> dict:
    return dict(zip(d[key_col], d[value_col]))

def make_codes_dict(d: pd.DataFrame, listname_col: str, key_col: str, value_col: str) -> dict[str, dict]:
    d_grpby = d.groupby(listname_col)
    return {
        list_name: make_dict(d_grpby.get_group(list_name), key_col, value_col)
        for list_name in d_grpby.groups
    }

def make_replace_dict(d: pd.DataFrame, var_col: str, listname_col: str, codes_dict: dict) -> dict[str, dict]:
    return {
        v: codes_dict[list_name]
        for v, list_name in zip(d[var_col], d[listname_col])
    }

def make_order_dict(d: pd.DataFrame, var_col: str, order_col: str, replace_dict: dict) -> dict[str, list]:
    return {
        v: [replace_dict[v][int(s)] for s in order.split()]
        for v, order in zip(d[var_col], d_hasorder[order_col])
    }

codes_dict = make_codes_dict(d_codes, "list_name", "code", "label")
replace_dict = make_replace_dict(d_vars.query("list_name.notna()"), "var", "list_name", codes_dict)
order_dict = make_order_dict(d_vars.query("order.notna()"), "var", "order", replace_dict)

# Đổi code thành label
d = d.replace(replace_dict)
# Tạo biến categorical và thiết lập thứ tự
for v in replace_dict:
    d[v] = pd.Categorical(d[v]) if v not in order_dict \
        else pd.Categorical(d[v], categories=order_dict[v], ordered=True)
# Đổi tên biến
d = d.rename(columns=make_dict(d_vars, "var", "label"))

d.head()

,STT,Giới,Tuổi,Trình độ học vấn,BMI,Trào ngược,Nóng rát,Đau ngực
0,1,Nam,38,Đại học trở lên,18.411116,Có,Có,Có
1,2,Nữ,43,Đại học trở lên,21.438402,Có,Có,Không
2,3,Nữ,21,Cấp 3,20.884253,Có,Có,Có
3,4,Nam,26,Đại học trở lên,20.912275,Có,Có,Có
4,5,Nữ,50,Cấp 2 trở xuống,20.355965,Có,Có,Có


---

[Bài trước](./15_categorical.ipynb) - [Danh sách bài](../README.md) - [Bài sau]()